In [1]:
import pandas as pd
import numpy as np


# Alpha 1: VOLUME_PRICE_CORRELATION
# 计算公式: (-1 * CORR(RANK(DELTA(LOG(VOLUME), 1)), RANK(((CLOSE - OPE N) / OPEN)), 6))
# 公式: -1 * 过去6天的VOLUME_DELTA与CLOSE_OPEN_RATIO的相关系数
# 该因子用于捕捉短期量价关系,当成交量变化与收盘价相关性较强时,可能预示未来价格将发生变化
def alpha_1(df):
    volume_delta = np.log(df['VOLUME']).diff(1)
    volume_delta_rank = volume_delta.rank(method='average')
    close_open_ratio = (df['CLOSE'] - df['OPEN']) / df['OPEN']
    close_open_ratio_rank = close_open_ratio.rank(method='average')
    corr = volume_delta_rank.rolling(6).corr(close_open_ratio_rank)
    return -1 * corr

In [3]:
data = pd.DataFrame({'VOLUME': [100, 200, 150, 300, 250, 151, 500, 510 , 600, 700],
                     'CLOSE': [10, 12, 15, 11, 13, 15, 11, 13, 15, 18],
                     'OPEN': [9, 10, 14, 10, 12, 14, 10, 12, 14, 10]})
print(alpha_1(data))

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
5         NaN
6   -0.854749
7   -0.939898
8   -0.702118
9   -0.395393
dtype: float64


In [4]:
# Alpha 2: PRICE_GAP
# 公式: -1 * 前一天的价格缺口
# 该因子用于捕捉市场短期动量,当前一交易日存在较大价格缺口时,可能预示未来价格将进一步变化
def alpha_2(df):
    delta = (((df['CLOSE'] - df['LOW']) - (df['HIGH'] - df['CLOSE'])) / (df['HIGH'] - df['LOW'])).diff(1)
    return -1 * delta

In [8]:
data = pd.DataFrame({'VOLUME': [100, 200, 150, 300, 250, 151, 500, 510 , 600, 700],
                     'CLOSE': [10, 12, 15, 11, 13, 15, 11, 13, 15, 18],
                     'OPEN': [9, 10, 14, 10, 12, 14, 10, 12, 14, 10],
                     'LOW':[10,12,15,14,13,15,14,13,15,14],
                     'HIGH':[4,5,6,7,8,9,10,11,12,13]})
print(alpha_2(data))

0         NaN
1   -0.000000
2   -0.000000
3   -0.857143
4    0.857143
5   -0.000000
6   -1.500000
7    1.500000
8   -0.000000
9    8.000000
dtype: float64


In [9]:
# Alpha 3: TRADING_MOMENTUM
# 公式: 过去6天收盘价与前一交易日最小值的正差值之和
# 该因子用于衡量股价的动量特征,当股价在较长时间内表现良好时,可能预示未来价格将继续上涨
def alpha_3(df):
    cond = (df['CLOSE'] == df['CLOSE'].shift(1))
    min_val = df['CLOSE'].where(df['CLOSE'] > df['CLOSE'].shift(1), df['LOW'].shift(1))
    max_val = df['CLOSE'].where(df['CLOSE'] <= df['CLOSE'].shift(1), df['HIGH'].shift(1))
    diff = df['CLOSE'] - min_val
    return diff.where(~cond, 0).rolling(6).sum()